In [1]:
!pip install spacy

In [2]:
!python -m spacy download en_core_web_lg

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [3]:
import spacy
import pickle
import random

In [4]:
text=pickle.load(open(r"C:\Users\Lenovo\Desktop\train_data.pkl","rb"))
print(text[0])

('Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [5]:
import re
def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data

In [6]:
import pandas as pd
import os
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin
from spacy.util import filter_spans

#nlp = spacy.blank("en") # load a new spacy model
nlp = spacy.load("en_core_web_sm") # load other spacy model

db = DocBin() # create a DocBin object

for text, annot in tqdm(trim_entity_spans(text[:10])): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    pat_orig = len(ents)
    filtered = filter_spans(ents) # THIS DOES THE TRICK
    pat_filt =len(filtered)
    doc.ents=filtered # label the text with the ents
    db.add(doc)


print("\nCONVERSION REPORT:")
print("Original number of patterns:", pat_orig)
print("Number of patterns after overlapping removal:", pat_filt)

os.chdir(r'C:\Users\Lenovo\Desktop\CV new model')
db.to_disk("./train.spacy")

100%|██████████| 10/10 [00:00<00:00, 82.45it/s]

Skipping entity

CONVERSION REPORT:
Original number of patterns: 22
Number of patterns after overlapping removal: 22


In [10]:
!python -m spacy init fill-config base_config.cfg config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [11]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

[+] Created output directory: output
[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    252.76    0.00    0.00    0.00    0.00
 20     200       6379.83  12340.78   96.26   96.77   95.74    0.96
 40     400        898.39    389.53  100.00  100.00  100.00    1.00
 60     600       1128.36    171.97   99.73   99.47  100.00    1.00
 80     800         95.86     40.21   99.73   99.47  100.00    1.00
100    1000         45.40     15.49  100.00  100.00  100.00    1.00
120    1200        148.71     25.83  100.00  100.00  100.00    1.00
140    1400        356.53    116.30  100.00  100.00 

[2022-09-24 20:47:48,257] [INFO] Set up nlp object from config
[2022-09-24 20:47:48,267] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-09-24 20:47:48,267] [INFO] Created vocabulary
[2022-09-24 20:47:48,267] [INFO] Finished initializing nlp object
[2022-09-24 20:47:49,665] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [9]:
!python -m spacy debug data --help

Usage: python -m spacy debug data [OPTIONS] CONFIG_PATH

  Analyze, debug and validate your training and development data. Outputs
  useful stats, and can help you find problems like invalid entity
  annotations, cyclic dependencies, low data labels and more.

  DOCS: https://spacy.io/api/cli#debug-data

Arguments:
  CONFIG_PATH  Path to config file  [required]

Options:
  -c, --code-path, --code PATH  Path to Python file with additional code
                                (registered functions) to be imported
  -IW, --ignore-warnings        Ignore warnings, only show stats and errors
  -V, --verbose                 Print additional information and explanations
  -NF, --no-format              Don't pretty-print the results
  --help                        Show this message and exit.
